In [2]:
!pip install llama-index --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install llama-index-llms-langchain --quiet
!pip install langchain_mistralai --quiet
!pip install langchain_community --quiet
!pip install mistralai --quiet
%pip install --upgrade --quiet langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 10.0 MB/s eta 0:00:00


In [3]:
from llama_index.core import Settings,VectorStoreIndex,SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_mistralai import ChatMistralAI

from google.colab import userdata
api_key=userdata.get('MISTRAL_AI_API_KEY')

Settings.embed_model= HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm=ChatMistralAI(model_name="mistral-large-latest",api_key=api_key,temperature=0.6,top_p=0.9,random_seed=42)
Settings.llm=llm


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
from langchain_community.utilities import SQLDatabase

db=SQLDatabase.from_



In [30]:
with open('DB/Chinook_sqlite.sql','r') as f:
  sql_script=f.read()
db=SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.get_usable_table_names())
sql_statement=sql_script.split(';')

for statment in sql_statement:
  if statment.strip():
    db.run(statment)



['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [27]:
db.get_usable_table_names()

[]

In [12]:
from langchain_core.messages import SystemMessage,HumanMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message=SystemMessage(content=SQL_PREFIX)

In [31]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit=SQLDatabaseToolkit(db=db,llm=llm)
tools=toolkit.get_tools()

In [32]:
from langgraph.prebuilt import create_react_agent
agent_executor=create_react_agent(llm,tools,messages_modifier=system_message)

<ipython-input-32-c92630159766>:2: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor=create_react_agent(llm,tools,messages_modifier=system_message)


In [69]:
import time
for s in agent_executor.stream({
    "messages":[HumanMessage(content="which customers are invoiced? dispaly only the first name concated with last name as full name")]
}):
  try:
    if str(s.get('agent').get('messages')[0]).split("'")[1]:
      print(str(s.get('agent').get('messages')[0]).split("'")[1])
    time.sleep(10)
    # print("------")
  except:
    pass

Here are the customers who are invoiced:\n\n- Luís Gonçalves\n- Leonie Köhler\n- François Tremblay\n- Bjørn Hansen\n- František Wichterlová


In [34]:
db.run('SELECT COUNT(DISTINCT CustomerId) FROM Invoice')

'[(59,)]'

In [76]:
from flask import Flask,request,jsonify
app = Flask(__name__)

@app.route('/webhook',methods=['POST'])
def webhook():
  re=request.get_json()
  intent=re.get('queryResult').get('queryText')
  response = ""
  for s in agent_executor.stream({"messages":[HumanMessage(content=intent)]}):
        try:
            if str(s.get('agent').get('messages')[0]).split("'")[1]:
                response = str(s.get('agent').get('messages')[0]).split("'")[1]
            time.sleep(1)
        except:
            pass

  return jsonify({
      "fulfillmentText":response
  })

if __name__=="__main__":
  app.run(debug=True,host='0.0.0.0',port=5000)





 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [74]:
!pip install pyngrok

In [75]:
!ngrok http 5000

http - start an HTTP tunnel

USAGE:
  ngrok http [address:port | port] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your a